# **目標一**:
### 使用 SQuAD 資料集和 facebook/bart-base 訓練 Question-Answering 模型 生成 Answer 方式


# 安裝套件

In [ ]:
pip install transformers datasets accelerate

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 325 kB 50.2 MB/s 
     |████████████████████████████████| 65 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 36.7 MB/s 
     |████████████████████████████████| 6.5 MB 36.5 MB/s 
     |████████████████████████████████| 895 kB 42.6 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 39.2 MB/s 
     |████████████████████████████████| 134 kB 48.4 MB/s 
     |████████████████████████████████| 212 kB 48.3 MB/s 
     |████████████████████████████████| 127 kB 39.4 MB/s 
     |████████████████████████████████| 271 kB 36.4 MB/s 
     |████████████████████████████████| 94 kB 373 kB/s 
     |████████████████████████████████| 144 kB 10.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    F

# 確認 GPU 分配

In [ ]:
!nvidia-smi

Tue Oct  5 15:03:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Mount 雲端硬碟

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## cd 到自己的雲端硬碟中的colab

In [ ]:
%cd /content/drive/"MyDrive"/"Colab Notebooks"  

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!ls

 BART_SQuAD
 BERT_DRCD
 Bert_drive
'BERT for QA and QG on DRCD dataset'
'「BERT for QG on DRCD dataset.ipynb」的副本'
 “BERT-NER_kashgari_512.ipynb
 BERT-NER_kashgari.ipynb
 BERT-NER_kashgari_Use.ipynb
'「BERT on 外賣資料集.ipynb」的副本'
 BERT_SQuAD
 dev-v1.1.json
 dg_graph.ipynb
 drcd
 Email_Summarization_for_Mbox_example.ipynb
'Email Summarization for Mbox.ipynb'
 FastText.ipynb
 FastText_patent_train.ipynb
 FastText_patent_use.ipynb
 “FastText_patent_use.ipynb”的副本
 glove.ipynb
 glove_training.ipynb
 LawMongoDB.ipynb
 Lyrics
 mp3_FOLDER
 PM2dot5_ExamDataCreater
'QG on drcd'
 testText2Speech.ipynb
 train-v1.1.json
 USPTO
 waimai
 Wiki_BuildClassifier.ipynb
 word2vec.ipynb
 word2vec_training.ipynb
 word_emb_sim_exp.ipynb
'成績判斷模型（A、B、C、D 、E 多分類）.ipynb 的副本'
 成績判斷模型（及格、不及格二分類）.ipynb


In [ ]:
!mkdir ./BART_SQuAD/
%cd ./BART_SQuAD/

mkdir: cannot create directory ‘./BART_SQuAD/’: File exists
/content/drive/MyDrive/Colab Notebooks/BART_SQuAD


In [ ]:
!ls

dev-v1.1.json  model  train-v1.1.json


# 資料前處理

## 下載資料集

In [ ]:
!cp /content/drive/Shareddrives/新手村/教學投影片/新生訓練/data/"SQuAD 1.1"/train-v1.1.json .
!cp /content/drive/Shareddrives/新手村/教學投影片/新生訓練/data/"SQuAD 1.1"/dev-v1.1.json .

### SQuAD 資料格式
Stanford 大學所整理的閱讀理解資料集 Stanford Question Answering Dataset (SQuAD) 
內容從維基百科中收集超過 10 萬筆的 CQA pair


For more information please refer to Paper: https://arxiv.org/abs/1606.05250

### Data format 資料格式

- version : <String> 資料集版本
- data : <Array>
  - title : <String> : 文章標題
  - id : <String> : 文章編號
  - paragraphs : <Array>
    - id : <String> : 文章編號_段落編號
    - context : <String> : 段落內容
    - qas : <Array>
      - question : <String> : 問題內容
      - id :<String> : 文章編號_段落編號_問題編號
      - answers : <Arrays>
        - answer_start : <int> text在文中位置
        - id : <String> : "1"表示為人工標註的答案，"2"以上為人工答題的答案
        - text : <string> : 答案內容


In [ ]:
import json
from pprint import pprint
with open('dev-v1.1.json') as file:
  train_data = json.load(file)

for ele in train_data['data']:
  pprint(ele['paragraphs'][0])
  break

{'context': 'Super Bowl 50 was an American football game to determine the '
            'champion of the National Football League (NFL) for the 2015 '
            'season. The American Football Conference (AFC) champion Denver '
            'Broncos defeated the National Football Conference (NFC) champion '
            'Carolina Panthers 24–10 to earn their third Super Bowl title. The '
            "game was played on February 7, 2016, at Levi's Stadium in the San "
            'Francisco Bay Area at Santa Clara, California. As this was the '
            '50th Super Bowl, the league emphasized the "golden anniversary" '
            'with various gold-themed initiatives, as well as temporarily '
            'suspending the tradition of naming each Super Bowl game with '
            'Roman numerals (under which the game would have been known as '
            '"Super Bowl L"), so that the logo could prominently feature the '
            'Arabic numerals 50.',
 'qas': [{'answers': [{'answe

## 讀取資料

In [ ]:
from pathlib import Path
def read_data(path, limit=None):
    path = Path(path)
    with open(path, 'rb') as f:
        data_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in data_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
                if limit != None and len(contexts) > limit:
                  return contexts, questions, answers
                  
    return contexts, questions, answers

In [ ]:
train_contexts, train_questions, train_answers = read_data('train-v1.1.json', 20000)
eval_contexts, eval_questions, eval_answers = read_data('dev-v1.1.json',2000)

In [ ]:
print(len(train_contexts))
print(len(eval_contexts))

20001
2002


In [ ]:
print(train_contexts[0])
print(train_questions[0])
print(train_answers[0])

Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}


# 將資料進行 Tokenize
## 將 input 資料轉換成 token id 、tpye_id 與 attention_mask

In [ ]:
from transformers import BartTokenizerFast, BartTokenizer, AutoConfig
tokenizer_fast = BartTokenizerFast.from_pretrained('facebook/bart-base')
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')
config = AutoConfig.from_pretrained('facebook/bart-base')

train_encodings = tokenizer_fast(train_contexts, train_questions, truncation=True, padding=True)
eval_encodings = tokenizer_fast(eval_contexts, eval_questions, truncation=True, padding=True)

## 檢查轉換是否正確

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
print(train_encodings['input_ids'][0])
print(tokenizer_fast.convert_ids_to_tokens(train_encodings['input_ids'][0]))
print(tokenizer_fast.decode(train_encodings['input_ids'][0]))

print(train_encodings['attention_mask'][0])
print(len(train_encodings['input_ids'][0]))

[0, 37848, 37471, 28108, 6, 5, 334, 34, 10, 4019, 2048, 4, 497, 1517, 5, 4326, 6919, 18, 1637, 31346, 16, 10, 9030, 9577, 9, 5, 9880, 2708, 4, 29261, 11, 760, 9, 5, 4326, 6919, 8, 2114, 24, 6, 16, 10, 7621, 9577, 9, 4845, 19, 3701, 62, 33161, 19, 5, 7875, 22, 39043, 1459, 1614, 1464, 13292, 4977, 845, 4130, 7, 5, 4326, 6919, 16, 5, 26429, 2426, 9, 5, 25095, 6924, 4, 29261, 639, 5, 32394, 2426, 16, 5, 7461, 26187, 6, 10, 19035, 317, 9, 9621, 8, 12456, 4, 85, 16, 10, 24633, 9, 5, 11491, 26187, 23, 226, 2126, 10067, 6, 1470, 147, 5, 9880, 2708, 2851, 13735, 352, 1382, 7, 6130, 6552, 625, 3398, 208, 22895, 853, 1827, 11, 504, 4432, 4, 497, 5, 253, 9, 5, 1049, 1305, 36, 463, 11, 10, 2228, 516, 14, 15230, 149, 155, 19638, 8, 5, 2610, 25336, 238, 16, 10, 2007, 6, 2297, 7326, 9577, 9, 2708, 4, 2, 2, 3972, 2661, 222, 5, 9880, 2708, 2346, 2082, 11, 504, 4432, 11, 226, 2126, 10067, 1470, 116, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## 新增 label 答案

In [ ]:
def add_label(encodings, answers):
    answers_lable_ids = []

    for answer in answers:
        label_tokens = tokenizer.tokenize(answer['text'] + tokenizer.eos_token, max_length=config.max_length, padding="max_length", truncation=True)
        lable_ids = tokenizer.convert_tokens_to_ids(label_tokens)
        answers_lable_ids.append(lable_ids)
    encodings.update({'labels': answers_lable_ids})

In [ ]:
add_label(train_encodings, train_answers)
add_label(eval_encodings, eval_answers)

In [ ]:
print(train_encodings['labels'][0])


[41742, 6552, 625, 3398, 208, 22895, 853, 1827, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# 定義 Dataset，並轉換成 tensor 格式

In [ ]:
import torch
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset = Dataset(train_encodings)
eval_dataset = Dataset(eval_encodings)

In [ ]:
train_dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0

# 載入模型架構( Seq2SeqLM )

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoConfig
config = AutoConfig.from_pretrained('facebook/bart-base')
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base', config=config)

## 查看模型架構

In [ ]:
print(model)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        

# 訓練模型

In [ ]:
import logging
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)

## 設定 epoch 與 batch size

In [ ]:
train_batch_size = 4      # 設定 training batch size
eval_batch_size = 4     # 設定 eval batch size
num_train_epochs = 1      # 設定 epoch

## 將資料丟入 DataLoader


In [ ]:
data_collator = default_data_collator
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

## Optimizer 、Learning rate 、Scheduler 設定

In [ ]:
learning_rate=3e-5          # 設定 learning_rate
gradient_accumulation_steps = 1   # 設定 幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },                                
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

max_train_steps 5001


## 將資料、參數丟入 Accelerator



In [ ]:
# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
!nvidia-smi

Tue Oct  5 12:34:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    73W / 149W |   1036MiB / 11441MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 開始訓練

In [ ]:
# Train!
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = 'model/'


total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch:": 0, "acc": 0 }

for epoch in trange(num_train_epochs, desc="Epoch"):
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    outputs = model(**batch)
    loss = outputs.loss
    loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      completed_steps += 1

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})

    if completed_steps >= max_train_steps:
      break
      
  # logger.info("***** Running eval *****")
  # model.eval()
  # for step, batch in enumerate(tqdm(eval_dataloader, desc="Eval Iteration")):
  #   outputs = model(**batch)
  #   predictions = outputs.logits.argmax(dim=-1)
  #   metric.add_batch(
  #       predictions=accelerator.gather(predictions),
  #       references=accelerator.gather(batch["labels"]),
  #   )

  # eval_metric = metric.compute()
  # logger.info(f"epoch {epoch}: {eval_metric}")
  # if eval_metric > best_epoch['acc']:
  #   best_epoch['epoch'] = num_train_epochs
  #   best_epoch['acc'] = eval_metric


  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + 'epoch_' + str(epoch) + '/', save_function=accelerator.save)


10/05/2021 12:34:59 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Use FP16 precision: False

10/05/2021 12:34:59 - INFO - __main__ - ***** Running training *****
10/05/2021 12:34:59 - INFO - __main__ -   Num examples = 20001
10/05/2021 12:34:59 - INFO - __main__ -   Num Epochs = 1
10/05/2021 12:34:59 - INFO - __main__ -   Instantaneous batch size per device = 4
10/05/2021 12:34:59 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) = 4
10/05/2021 12:34:59 - INFO - __main__ -   Gradient Accumulation steps = 1
10/05/2021 12:34:59 - INFO - __main__ -   Total optimization steps = 5001


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5001 [00:00<?, ?it/s]

{'epoch': 0, 'step': 0, 'loss': 15.458333969116211}
{'epoch': 0, 'step': 50, 'loss': 4.10726261138916}
{'epoch': 0, 'step': 100, 'loss': 1.9152328968048096}
{'epoch': 0, 'step': 150, 'loss': 1.2193603515625}
{'epoch': 0, 'step': 200, 'loss': 0.9062670469284058}
{'epoch': 0, 'step': 250, 'loss': 1.1943349838256836}
{'epoch': 0, 'step': 300, 'loss': 0.722119927406311}
{'epoch': 0, 'step': 350, 'loss': 0.894058108329773}
{'epoch': 0, 'step': 400, 'loss': 0.8700487017631531}
{'epoch': 0, 'step': 450, 'loss': 0.7099778652191162}
{'epoch': 0, 'step': 500, 'loss': 0.7241147756576538}
{'epoch': 0, 'step': 550, 'loss': 0.5593327283859253}
{'epoch': 0, 'step': 600, 'loss': 0.7018052339553833}
{'epoch': 0, 'step': 650, 'loss': 0.5605195164680481}
{'epoch': 0, 'step': 700, 'loss': 0.790673553943634}
{'epoch': 0, 'step': 750, 'loss': 0.5202854871749878}
{'epoch': 0, 'step': 800, 'loss': 0.5174151659011841}
{'epoch': 0, 'step': 850, 'loss': 0.4338461756706238}
{'epoch': 0, 'step': 900, 'loss': 0.595

# 分析模型 (計算 exact match, F1-score )

In [ ]:
from transformers import BartTokenizerFast, AutoModelForSeq2SeqLM, AutoConfig, default_data_collator
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%cd /content/drive/"MyDrive"/"Colab Notebooks"/BART_SQuAD/
!ls

/content/drive/MyDrive/Colab Notebooks/BART_SQuAD
dev-v1.1.json  model  train-v1.1.json


## 載入模型與測試資料

In [ ]:
tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-base")
config = AutoConfig.from_pretrained("./model/epoch_0/config.json") 
model = AutoModelForSeq2SeqLM.from_pretrained("./model/epoch_0/pytorch_model.bin", config=config)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

In [ ]:
eval_contexts, eval_questions, eval_answers = read_data('dev-v1.1.json',2000)
eval_encodings = tokenizer(eval_contexts, eval_questions, truncation=True, padding=True)
add_label(eval_encodings, eval_answers)
eval_dataset = Dataset(eval_encodings)

In [ ]:
eval_batch_size = 12      # 設定 batch size
data_collator = default_data_collator

eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, eval_dataloader = accelerator.prepare(
    model, eval_dataloader
)

## 預測資料

In [ ]:
print("***** Running eval *****")
model.eval()
ref = []
pre = []
index = 0

for step, batch in enumerate(tqdm(eval_dataloader, desc="Eval Iteration")):
  output_ids = model.generate(batch['input_ids'], num_beams=3, max_length=20, early_stopping=True)
  predictions = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in output_ids]
  answers = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in batch['labels']]

  for prediction, answer in zip(predictions ,answers):
    
    ref.append({'answers': {'answer_start': [-100], 'text': [answer]}, 'id': str(index)})
    pre.append({'prediction_text': prediction, 'id': str(index)})
    index+=1


***** Running eval *****


Eval Iteration:   0%|          | 0/167 [00:00<?, ?it/s]

In [ ]:
print(ref[0])
print(pre[0])

{'answers': {'answer_start': [-100], 'text': ['Denver Broncos']}, 'id': '0'}
{'prediction_text': 'Denver Broncos', 'id': '0'}


In [ ]:
import datasets

squad_metric = datasets.load_metric("squad")
results = squad_metric.compute(predictions=pre, references=ref)
print(results)

{'exact_match': 63.786213786213786, 'f1': 76.1832356787403}


# Inference

In [ ]:
# **撰寫預測程式**
def QA_model(model, context, question):

  input_encodings = tokenizer([context], [question], truncation=True, padding=True)
  input_dataset = Dataset(input_encodings)

  data_collator = default_data_collator
  input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)  

  accelerator = Accelerator()
  model, input_dataloader = accelerator.prepare(model, input_dataloader)
  for batch in input_dataloader:
    output_ids = model.generate(**batch, num_beams=3, max_length=20, early_stopping=True)
    predictions_text = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in output_ids]

  return predictions_text

In [ ]:
context = '''Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's struggle against Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic and subjugate all wizards and Muggles (non-magical people).'''
question = 'Who is the author of harry potter?'

answer = QA_model(model, context, question)  
print(answer)

['J. K. Rowling']


# **目標二**:
### 使用 SQuAD 資料集和 facebook/bart-base 訓練 Question-Generation 模型
#### input：context, answer
#### ouput：question


In [ ]:
%cd ./BART_SQuAD/
!ls

[Errno 2] No such file or directory: './BART_SQuAD/'
/content/drive/MyDrive/Colab Notebooks/BART_SQuAD
dev-v1.1.json  model  train-v1.1.json


In [ ]:
# 使用 20000 筆 training data，與 2000 筆 testing data 訓練
import json
from pprint import pprint
with open('train-v1.1.json') as file:
  train_data = json.load(file)

for ele in train_data['data']:
  pprint(ele['paragraphs'][0])
  break

{'context': 'Architecturally, the school has a Catholic character. Atop the '
            "Main Building's gold dome is a golden statue of the Virgin Mary. "
            'Immediately in front of the Main Building and facing it, is a '
            'copper statue of Christ with arms upraised with the legend '
            '"Venite Ad Me Omnes". Next to the Main Building is the Basilica '
            'of the Sacred Heart. Immediately behind the basilica is the '
            'Grotto, a Marian place of prayer and reflection. It is a replica '
            'of the grotto at Lourdes, France where the Virgin Mary reputedly '
            'appeared to Saint Bernadette Soubirous in 1858. At the end of the '
            'main drive (and in a direct line that connects through 3 statues '
            'and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
                       'text': 'Saint Bernadette Soubirous'}],
          'id': '5733be284776f41

# 訓練模型

# 分析模型，計算 token scores（BLUE、ROUGE)

# Inference